In [1]:
# 1:
import paho.mqtt.client as mqtt
import threading
import os
import re
# import motionlocalizer.settings
import json
# import requests
from shapely.geometry import Polygon
from shapely.ops import cascaded_union
import xml.etree.ElementTree
import pprint
import numpy as np
import matplotlib.pyplot as plt
# import matplotlib.pylab as plt
import matplotlib
from matplotlib.patches import Circle
import time as pythonTimer
import random
from random import randint
import scipy.stats as stats
from scipy.stats import multivariate_normal
import csv
import datetime
import pandas as pd
import datetime
import pandas as pd
import numpy as np

global simworldname
global groundtruth
global motion_readings
global my_map
global grid_map
global distances
global oracle
global analyzer
global dataset_times
global dataset_readings
global dataset
global motions_order
global motion_sensors
global motions_sensor_data
global simulated_sensor_readings
global agentsnum
global sensors_list
global estimotes_list
global sr_matrix
global indices_to_keep
global gtx
global gty
global gtx2
global gty2
global rsr
global df_
global gtt
global human_readable_sensor_array
global sensed_areas
global plotflag
global syntheticTrace

In [ ]:
agentsnum = 1
plotflag = True
sensed_areas = {}
human_readable_sensor_array = []
gtt = []
gtx = []
gty = []
gtx2 = []
gty2 = []
rsr = []
indices_to_keep = [0, 1, 2, 3, 4, 5, 11, 15, 16, 17, 18, 19, 20, 21]
simworldname = 'Data//simulationWorld2.xml' #ENVIROMENT
# groundtruth = 'Data//28jun2.csv' #AGENT'S MOVEMENTS

estimote_readings = 

motion_readings = 'Data//Sensor Readings//4Jul1_events.txt'
datasetname = 'Data//Pandas Datasets//Dataset_4Jul12'
syntheticTrace = 'Data//Synthetic Agent Traces//AgentTrace_4Jul12.csv'
syntheticTrace_edited = 'Data//Synthetic Agent Traces//(Edited)AgentTrace_4Jul12.csv'

# syntheticTrace2 = 'Data//Synthetic Agent Traces//AgentTrace_4Jul12p.csv'

# groundtruth = 'AgentDetailedTasksFile.csv' #AGENT'S MOVEMENTS

my_map = [(0.0, 0.0), (0.0, 10.6), (6.6, 10.6), (6.6, 0.0)]
grid_map = np.zeros((int(my_map[2][0]) * 10, int(my_map[2][1]) * 10))
distances = []

oracle = {}
oracle['time'] = []
oracle['location'] = []

analyzer = {}
analyzer['time'] = []
analyzer['gridmap'] = []
dataset_times = []
dataset_readings = []
dataset = ['time', 'gt(x)', 'gt(y)', 'motion_readings']

motions_order = [1, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 2, 20, 21, 22, 3, 4, 5, 6, 7, 8, 9]
motion_sensors = [1, 10, 11, 12, 13, 14, 2, 3, 4, 5, 6, 7, 8, 9]
motions_sensor_data = {}
simulated_sensor_readings = []
sensors_list = []
estimotes_list = []
sr_matrix = np.zeros([1, 14])



In [2]:
def SensorReadingInterpretation(dataset_readings):
    templist = [[0] * 14]
    for i in range(0, len(dataset_readings)):
        numlist = [int(s) for s in dataset_readings[i] if s.isdigit()]
        numlist = [numlist[j] for j in range(0, len(numlist)) if j in indices_to_keep]
        templist.append(numlist)
        
    templist_modified = [[0] * 14]
    for row in range(1, len(templist)):
        history = templist_modified[row - 1]
        row_modified = [0] * 14
        for i in range(len(history)):    
            if (templist[row][i] == 0 and history[i] == 1):
                row_modified[i] = 1
            
            elif (templist[row][i] == 0 and history[i] == 0):
                row_modified[i] = 0

            elif (templist[row][i] == 1 and history[i] == 1):
                row_modified[i] = 0

            elif (templist[row][i] == 1 and history[i] == 0):
                row_modified[i] = 1

        templist_modified.append(row_modified)

    del templist_modified[0]
    return templist

def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    # print(value - array[idx])
    return idx

In [ ]:
# sensor_readings = open(motion_readings, "r")
sensor_readings = open(motion_readings, "r").readlines()

#----- Extracting sensors reading from the groundtruth dataset    
for i in range(0, len(sensor_readings)):
    time = sensor_readings[i].split(';')[0]
    sensor_array = sensor_readings[i].split(';')[1]
    dataset_times.append(int(time))
    dataset_readings.append(sensor_array)

#----- Convert sensor readings to human readable format
# We do not change the values since the job has already been done by Parisa
human_readable_sensor_array = SensorReadingInterpretation(dataset_readings)


In [ ]:
with open(syntheticTrace, 'r') as agent_data, open(syntheticTrace_edited, 'w', newline='') as output:
    writer = csv.writer(output)
    reader = csv.reader(agent_data, delimiter = ',', quotechar = ',')
    counter = 0
    for row in reader:
        if (reader.line_num == 1):
            writer.writerow(row)
            prev_row = row
        else:
            if (row[5] == 'walk' and row[2] == prev_row[2] and row[3] == prev_row[3]):
                counter += 1
            else:
                writer.writerow(row)
            prev_row = row

In [ ]:
syntheticTrace = syntheticTrace_edited

In [ ]:
temp_gtx = []
temp_gty = []
temp_gtt = []

temp_gtx2 = []
temp_gty2 = []

counter = 0
with open(syntheticTrace, 'r') as agent_data:
    reader = csv.reader(agent_data, delimiter = ',', quotechar = ',')
    line = 0
    for row in reader:
        temp_gtt.append(float(row[0]))
        temp_gtx.append(6.6 - float(row[2]))
        temp_gty.append(float(row[3]))
        counter += 1

if (agentsnum == 2):
    counter = 0
    with open(syntheticTrace2, 'r') as agent_data:
        reader = csv.reader(agent_data, delimiter = ',', quotechar = ',')
        line = 0
        for row in reader:
            new_row = []
            temp_gtx2.append(6.6 - float(row[2]))
            temp_gty2.append(float(row[3]))
            counter += 1

        
gtx.append(temp_gtx)
gty.append(temp_gty)
gtt.append(temp_gtt)

if (agentsnum == 2):
    gtx2.append(temp_gtx2)
    gty2.append(temp_gty2)


gtt = dataset_times
gtx = gtx[0]
gty = gty[0]

if (agentsnum == 2):
    gtx2 = gtx2[0]
    gty2 = gty2[0]
    

In [ ]:
for i in range(0, len(gtt)):
    ind = find_nearest(dataset_times, int(gtt[i]))
    rsr.append(human_readable_sensor_array[ind])  

In [ ]:
if (agentsnum == 2):
    columns = ['time', 'gtx', 'gty', 'gtx2', 'gty2', 'gt_motion_readings', 'synthetic_motion_readings']
    
else:
    columns = ['time', 'gtx', 'gty', 'gt_motion_readings', 'synthetic_motion_readings']

index = range(0, len(gtx))    
df_ = pd.DataFrame(index=index, columns=columns)
    
df_.time = dataset_times
df_.gtx = gtx
df_.gty = gty
df_.gt_motion_readings = rsr
    
# df_.time = gtt
# df_.gtx = gtx[0:len(gtt)]
# df_.gty = gty[0:len(gtt)]
# df_.gt_motion_readings = rsr

# if (agentsnum == 2):
#     df_.gtx2 = gtx2[0:len(gtt)]
#     df_.gty2 = gty2[0:len(gtt)]

    

In [ ]:
len(dataset_times)

In [ ]:
df_.to_csv(datasetname + ".csv", sep=',', index=False)

In [ ]:


datasetname12 = 'Data//Pandas Datasets//Dataset_4Jul12'
datasetname11 = 'Data//Pandas Datasets//Dataset_4Jul11'
datasetname1  = 'Data//Pandas Datasets//Dataset_4Jul1'

datasetname21 = 'Data//Pandas Datasets//Dataset_4Jul21'
datasetname22 = 'Data//Pandas Datasets//Dataset_4Jul22'
datasetname2  = 'Data//Pandas Datasets//Dataset_4Jul2'

df_12 = (pd.read_csv(datasetname12 + ".csv"))
df_11 = (pd.read_csv(datasetname11 + ".csv"))
df_22 = (pd.read_csv(datasetname22 + ".csv"))
df_21 = (pd.read_csv(datasetname21 + ".csv"))

columns = ['time', 'gtx', 'gty', 'gtx2', 'gty2', 'gt_motion_readings', 'synthetic_motion_readings']
index = range(0, len(df_12))    
df_1 = pd.DataFrame(index=index, columns=columns)
df_1.time = df_11.time
df_1.gtx = df_11.gtx
df_1.gty = df_11.gty
df_1.gtx2 = df_12.gtx
df_1.gty2 = df_12.gty
df_1.gt_motion_readings = df_11.gt_motion_readings
df_1.synthetic_motion_readings = df_11.synthetic_motion_readings
df_1.to_csv(datasetname1 + ".csv", sep=',', index=False)

df_2 = pd.DataFrame(index=index, columns=columns)
df_2.time = df_21.time
df_2.gtx = df_21.gtx
df_2.gty = df_21.gty
df_2.gtx2 = df_22.gtx
df_2.gty2 = df_22.gty
df_2.gt_motion_readings = df_21.gt_motion_readings
df_2.synthetic_motion_readings = df_21.synthetic_motion_readings
df_2.to_csv(datasetname2 + ".csv", sep=',', index=False)

In [19]:
def AddEstimoteActivation(D, path, estimote_readings):
    df_ = (pd.read_csv(path + D + ".csv"))
    df_.loc[:,'gt_estimote_readings'] = pd.Series(index=df_.index)
    df_.loc[:,'gt_ids'] = pd.Series(index=df_.index)
    df_.loc[:,'synthetic_estimote_readings'] = pd.Series(index=df_.index)
    df_.loc[:,'synthetic_ids'] = pd.Series(index=df_.index)
    
    for time_index in range(len(estimote_readings['time'])):
        ind = find_nearest(df_.time, estimote_readings['time'][time_index])
        df_.gt_estimote_readings[ind] = str(estimote_readings['data'][time_index])
        df_.gt_ids[ind] = str(estimote_readings['ids'][time_index])
    
    df_.gt_estimote_readings.fillna(str([0] * len(estimote_readings['data'][0])), inplace = True)
    df_.gt_ids.fillna("[]", inplace = True)
    df_.to_csv(path + D + "_W_Estimotes" + ".csv", sep=',', index=False)
    
def GetEstimoteReadings(E, D, estimote_path, dataset_path):
    df_ = (pd.read_csv(dataset_path + D + ".csv"))
    readings = {}
    readings['time'] = []
    readings['data'] = []
    readings['ids'] = []
    
    Estimote_names = ["n1-bed", "n1-dog", "n1-chair", "n2-fridge", "n2-bed", "n2-bag", "n2-bike", "n2-shoe", "n2-dog", 
                      "n2-generic", "n2-chair", "n2-car", "n3-generic", "n3-bag", "n3-fridge", "n3-dog", "n3-bed", 
                      "n3-chair", "n3-bike", "n3-door", "n3-car", "n3-shoe", "n4-generic", "n4-bag", "n4-fridge",
                      "n4-dog", "n4-chair", "n4-bike", "n4-door", "n4-car", "n4-shoe"]
    
    with open(estimote_path + E, 'r') as estimote_data:
        reader = csv.reader(estimote_data, delimiter = ',', quotechar = ',')
        line = 0
        fired_sensors = []
        ids = []
        previous_time = -1
        for row in reader:
            if (reader.line_num == 1):
                time_interval = abs(df_.time[0] - int(row[0]))
                fired_sensors.append(row[2].split(':')[1].replace(' ', ''))
                
            if (previous_time == row[0]):
                fired_sensors.append(row[2].split(':')[1].replace(' ', ''))
                if row[1].replace(' ', '') not in ids: 
                    ids.append(row[1].replace(' ', ''))
            else:
                gt_estimotes = [0] * len(Estimote_names)
                for sensor in fired_sensors:
                    if (sensor in Estimote_names):
                        ind = Estimote_names.index(sensor)
                        gt_estimotes[ind] = 1
                
                if (sum(gt_estimotes) > 0):
                    readings['time'].append(int(previous_time) + time_interval)
                    readings['data'].append(gt_estimotes)
                    readings['ids'].append(ids)
                    
                fired_sensors = []
                ids = []
                previous_time = row[0]
                fired_sensors.append(row[2].split(':')[1].replace(' ', ''))
                if row[1].replace(' ', '') not in ids: 
                    ids.append(row[1].replace(' ', ''))
                
    return readings

In [20]:
D1 = "Dataset_28Jun1_edited"
D2 = "Dataset_28Jun2_edited"
D3 = "Dataset_4Jul2"
D4 = "Dataset_4Jul1"
dataset_path = "Data//Pandas Datasets//"

E1 = "28Jun1_events.txt"
E2 = "28Jun2_events.txt" 
E3 = "4Jul1_events.txt"
E4 = "4Jul2_events.txt"
estimote_path = "Data//Estimote Readings//"

Ds = [D1, D2, D3, D4]
Es = [E1, E2, E3, E4]

for index in range(len(Ds)):
    estimote_readings = GetEstimoteReadings(Es[index], Ds[index], estimote_path, dataset_path)
    AddEstimoteActivation(Ds[index], dataset_path, estimote_readings)

C:\Users\shada\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Users\shada\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
